In [1]:
!pip install flask

In [2]:
from flask import Flask, request, jsonify
import random
import json


In [ ]:
# Load intents from JSON
# with open("intents.json") as f:
#     intents = json.load(f)["intents"]

# Quick check
# for intent in intents:
#     print(intent["tag"], "->", intent["patterns"])


In [ ]:
# app = Flask(__name__)

# @app.route("/")
# def home():
#     return "🤖 Flask Chatbot is Running!"

# @app.route("/chat", methods=["POST"])
# def chat():
#     data = request.get_json()
#     user_message = data.get("message", "").lower()

#     for intent in intents:
#         for pattern in intent["patterns"]:
#             if pattern in user_message:
#                 response = random.choice(intent["responses"])
#                 return jsonify({"reply": response})

#     return jsonify({"reply": "I'm not sure I understand. Can you rephrase?"})


In [ ]:
# from threading import Thread

# def run():
#     app.run(host="127.0.0.1", port=5000)

# thread = Thread(target=run)
# thread.start()


In [3]:
!pip install torch --quiet

In [4]:
!pip install transformers --quiet

In [5]:
!pip install accelerate --quiet

In [6]:
pip install --upgrade transformers torch huggingface_hub[hf_xet]

  Using cached huggingface_hub-1.0.1-py3-none-any.whl.metadata (13 kB)
INFO: pip is looking at multiple versions of huggingface-hub[hf-xet] to determine which version is compatible with other requirements. This could take a while.
  Using cached huggingface_hub-1.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl.metadata (5.0 kB)
Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl (2.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import nest_asyncio
from threading import Thread
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

nest_asyncio.apply()  # allows Flask to run in Jupyter

app = Flask(__name__)

# 🔹 Load a tiny pretrained conversational model
MODEL_NAME = "microsoft/DialoGPT-small"  # about 350MB
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    user_input = data.get("message", "")

    # Encode and generate response
    inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
    reply_ids = model.generate(
        inputs,
        max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
    )
    reply = tokenizer.decode(reply_ids[:, inputs.shape[-1]:][0], skip_special_tokens=True)
    return jsonify({"response": reply})

# 🔹 Run Flask in background thread
def run_app():
    app.run(port=5050, debug=False)

thread = Thread(target=run_app)
thread.start()


e:\Learn\Python\ChatBot\Model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
import sys

# Optional: color printing (works on most terminals)
class Log:
    @staticmethod
    def info(msg): print(f"\033[94m[INFO]\033[0m {msg}")
    @staticmethod
    def success(msg): print(f"\033[92m[SUCCESS]\033[0m {msg}")
    @staticmethod
    def warning(msg): print(f"\033[93m[WARNING]\033[0m {msg}")
    @staticmethod
    def error(msg): print(f"\033[91m[ERROR]\033[0m {msg}")

def progress(msg):
    for dot in "....":
        sys.stdout.write(f"\r{msg}{dot}")
        sys.stdout.flush()
        time.sleep(0.2)
    print()  # newline

# -----------------------------
# CONFIG
# -----------------------------
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# -----------------------------
# PHASE 1: Load Tokenizer
# -----------------------------
Log.info("Initializing tokenizer loading...")
try:
    progress("Loading tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    Log.success("Tokenizer loaded successfully ✅")
except Exception as e:
    Log.error(f"Tokenizer loading failed ❌: {e}")
    sys.exit(1)

# -----------------------------
# PHASE 2: Determine Device
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
Log.info(f"Device selected: {device.upper()}")

# -----------------------------
# PHASE 3: Load Model
# -----------------------------
Log.info("Loading model weights (this may take several minutes)...")
try:
    progress("Downloading / Initializing model")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    model.to(device)
    Log.success("Model loaded successfully ✅")
except Exception as e:
    Log.error(f"Model loading failed ❌: {e}")
    sys.exit(1)

# -----------------------------
# PHASE 4: Define Generation Function
# -----------------------------
def generate_response(user_input):
    Log.info(f"Generating response for: '{user_input}'")
    try:
        inputs = tokenizer(user_input, return_tensors="pt").to(device)
        start = time.time()
        outputs = model.generate(**inputs, max_new_tokens=100)
        end = time.time()
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        Log.success(f"Response generated in {end - start:.2f} seconds ✅")
        return response
    except Exception as e:
        Log.error(f"Error during generation ❌: {e}")
        return "Error generating response."

# -----------------------------
# PHASE 5: Test Run
# -----------------------------
Log.info("Starting test prompt...")
test_text = "Hello, how are you?"
response = generate_response(test_text)

print("\n==============================")
print(f"User Input   : {test_text}")
print(f"Model Output : {response}")
print("==============================\n")

Log.success("All steps completed successfully 🎉")


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    user_message = data.get("message", "")
    bot_reply = generate_response(user_message)
    return jsonify({"reply": bot_reply})

# Run Flask app in background thread (same as before)
from threading import Thread

def run():
    app.run(host="127.0.0.1", port=5050)

thread = Thread(target=run)
thread.start()